API URLs 

shootings - https://services1.arcgis.com/79kfd2K6fskCAkyg/arcgis/rest/services/Gun_Violence_Data/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson



In [20]:
import pandas as pd
import sqlite3
import os
import requests
import json

# Imports

In [21]:
# # # -------------------------------------------------------- #
# # # Only run the API calls if you dont have the current data #
# # # -------------------------------------------------------- #

# import pandas as pd
# import os
# import requests
# import json


# url = 'https://services1.arcgis.com/79kfd2K6fskCAkyg/arcgis/rest/services/Gun_Violence_Data/FeatureServer/0/query' # noqa

# batch_size = 1000
# offset = 0
# data_list = []

# params = {
#     'where': '1=1',
#     'outFields': '*',
#     'returnGeometry': 'false',
#     'f': 'json',
#     'resultOffset': offset,
#     'resultRecordCount': batch_size
# }

# while True:
#     params['resultOffset'] = offset
#     response = requests.get(url, params=params)

#     print(f"Requesting URL: {response.url}")

#     if response.status_code != 200:
#         print(f"Error {response.status_code}: {response.text}")
#         break

#     try:
#         query_result = response.json()
#     except ValueError:
#         print(f"Failed to parse JSON response: {response.text}")
#         break

#     features = query_result.get('features', [])

#     if not features:
#         break

#     for feature in features:
#         data_list.append(feature['attributes'])

#     if len(features) < batch_size:
#         break

#     offset += batch_size

# if data_list:
#     df = pd.DataFrame(data_list)

#     output_directory = '../../data'

#     os.makedirs(output_directory, exist_ok=True)

#     output_path = os.path.join(output_directory, 'shootings.csv')
#     df.to_csv(output_path, index=False)

#     print(f"Data saved to {output_path}")
# else:
#     print("No data retrieved.")
    

# Read in data 

In [22]:
data = pd.read_csv('../../data/shootings.csv')
data.head()

,Case_Number,DateTime,Address,Neighborhood,Council_District,Latitude,Longitude,ZIP_Code,Crime_Type,Cause,Sex,Race,Age_Group,Division_Name,ObjectId
0,8010097010,1291773600000,"N 33rd St @ Bank St, Louisville, Ky",Portland,5.0,38.272650,-85.800687,40212,Non-Fatal Shooting,Gun Shot Wound,MALE,WHITE,25-34,First,1
1,8010097306,1291867200000,"Dixie Hwy @ W Gaulbert Ave, Louisville, Ky",Algonquin,6.0,38.227553,-85.790800,40210,Homicide,Gun Shot Wound,MALE,BLACK,18-24,Second,2
2,8010097982,1292104800000,"1700 Block Of Patton Ct, Louisville, Ky",Algonquin,6.0,38.224834,-85.778739,40210,Non-Fatal Shooting,Gun Shot Wound,MALE,BLACK,18-24,Second,3
3,8010098167,1292155200000,"S 30th St @ W Muhammad Ali Blvd, Louisville, Ky",Russell,5.0,38.255901,-85.800531,40212,Homicide,Gun Shot Wound,MALE,BLACK,25-34,First,4
4,8010099052,1292443200000,"3600 Block Of Manslick Rd, Louisville, Ky",Jacobs,3.0,38.195354,-85.799650,40215,Homicide,Blunt force Trauma,MALE,WHITE,1-10,Fourth,5


In [23]:
data['DateTime'] = pd.to_datetime(data['DateTime'], unit='ms').dt.normalize()
data['DateTime'] = data['DateTime'].dt.strftime('%Y-%m-%d %H:%M:%S')
data['Date'] = data['DateTime'].str[:10]


data.head()

,Case_Number,DateTime,Address,Neighborhood,Council_District,Latitude,Longitude,ZIP_Code,Crime_Type,Cause,Sex,Race,Age_Group,Division_Name,ObjectId,Date
0,8010097010,2010-12-08 00:00:00,"N 33rd St @ Bank St, Louisville, Ky",Portland,5.0,38.272650,-85.800687,40212,Non-Fatal Shooting,Gun Shot Wound,MALE,WHITE,25-34,First,1,2010-12-08
1,8010097306,2010-12-09 00:00:00,"Dixie Hwy @ W Gaulbert Ave, Louisville, Ky",Algonquin,6.0,38.227553,-85.790800,40210,Homicide,Gun Shot Wound,MALE,BLACK,18-24,Second,2,2010-12-09
2,8010097982,2010-12-11 00:00:00,"1700 Block Of Patton Ct, Louisville, Ky",Algonquin,6.0,38.224834,-85.778739,40210,Non-Fatal Shooting,Gun Shot Wound,MALE,BLACK,18-24,Second,3,2010-12-11
3,8010098167,2010-12-12 00:00:00,"S 30th St @ W Muhammad Ali Blvd, Louisville, Ky",Russell,5.0,38.255901,-85.800531,40212,Homicide,Gun Shot Wound,MALE,BLACK,25-34,First,4,2010-12-12
4,8010099052,2010-12-15 00:00:00,"3600 Block Of Manslick Rd, Louisville, Ky",Jacobs,3.0,38.195354,-85.799650,40215,Homicide,Blunt force Trauma,MALE,WHITE,1-10,Fourth,5,2010-12-15


In [24]:
data['Council_District'] = data['Council_District'].astype('Int64')
data['ZIP_Code'] = data['ZIP_Code'].astype(str)
text_cols = [
    'Neighborhood', 'Crime_Type', 'Cause',
    'Sex', 'Race', 'Age_Group', 'Division_Name'
]

for col in text_cols:
    data[col] = data[col].str.strip().str.title()
    
data[['Cause', 'Sex', 'Race', 'Age_Group']] = (
    data[['Cause', 'Sex', 'Race', 'Age_Group']]
    .fillna('Unknown')
)

In [25]:
data.insert(1, 'Date', data.pop('Date'))
data.insert(0, 'ObjectId', data.pop('ObjectId'))

In [26]:
data.head()

,ObjectId,Case_Number,Date,DateTime,Address,Neighborhood,Council_District,Latitude,Longitude,ZIP_Code,Crime_Type,Cause,Sex,Race,Age_Group,Division_Name
0,1,8010097010,2010-12-08,2010-12-08 00:00:00,"N 33rd St @ Bank St, Louisville, Ky",Portland,5,38.272650,-85.800687,40212,Non-Fatal Shooting,Gun Shot Wound,Male,White,25-34,First
1,2,8010097306,2010-12-09,2010-12-09 00:00:00,"Dixie Hwy @ W Gaulbert Ave, Louisville, Ky",Algonquin,6,38.227553,-85.790800,40210,Homicide,Gun Shot Wound,Male,Black,18-24,Second
2,3,8010097982,2010-12-11,2010-12-11 00:00:00,"1700 Block Of Patton Ct, Louisville, Ky",Algonquin,6,38.224834,-85.778739,40210,Non-Fatal Shooting,Gun Shot Wound,Male,Black,18-24,Second
3,4,8010098167,2010-12-12,2010-12-12 00:00:00,"S 30th St @ W Muhammad Ali Blvd, Louisville, Ky",Russell,5,38.255901,-85.800531,40212,Homicide,Gun Shot Wound,Male,Black,25-34,First
4,5,8010099052,2010-12-15,2010-12-15 00:00:00,"3600 Block Of Manslick Rd, Louisville, Ky",Jacobs,3,38.195354,-85.799650,40215,Homicide,Blunt Force Trauma,Male,White,1-10,Fourth


# Generate Database

```sql
-- Table 1: Address
CREATE TABLE IF NOT EXISTS Address (
    ObjectId INTEGER PRIMARY KEY,
    Address TEXT,
    Neighborhood TEXT,
    ZIP_Code TEXT
);

-- Table 2: CaseInfo
CREATE TABLE IF NOT EXISTS CaseInfo (
    ObjectId INTEGER PRIMARY KEY,
    Date TEXT,
    Case_Number TEXT,
    Division_Name TEXT,
    Council_District INTEGER,
    Crime_Type TEXT,
    Cause TEXT
);

-- Table 3: Demographics
CREATE TABLE IF NOT EXISTS Demographics (
    ObjectId INTEGER PRIMARY KEY,
    Age_Group TEXT,
    Sex TEXT,
    Race TEXT
);

-- Table 4: Geo
CREATE TABLE IF NOT EXISTS Geo (
    ObjectId INTEGER PRIMARY KEY,
    Latitude REAL,
    Longitude REAL
);
```

In [30]:
# Connect to SQLite
conn = sqlite3.connect('../../database/crime_data.db')

# Insert Address table
address_cols = ['Address', 'Neighborhood', 'ZIP_Code']
data[['ObjectId'] + address_cols].to_sql(
    'Address',
    conn,
    if_exists='replace',
    index=False
)

# Insert CaseInfo table
caseinfo_cols = ['Date', 'Case_Number', 'Division_Name', 'Council_District', 'Crime_Type', 'Cause']
data[['ObjectId'] + caseinfo_cols].to_sql(
    'CaseInfo',
    conn,
    if_exists='replace',
    index=False
)

# Insert Demographics table
demo_cols = ['Age_Group', 'Sex', 'Race']
data[['ObjectId'] + demo_cols].to_sql(
    'Demographics',
    conn,
    if_exists='replace',
    index=False
)

# Insert Geo table
geo_cols = ['Latitude', 'Longitude']
data[['ObjectId'] + geo_cols].to_sql(
    'Geo',
    conn,
    if_exists='replace',
    index=False
)

# Close connection
conn.close()


In [29]:
import json
import os


def generate_toc_from_notebook(notebook_path):
    """
    Parses a local .ipynb file and generates Markdown for a Table of Contents.
    """
    if not os.path.isfile(notebook_path):
        print(f"❌ Error: File not found at '{notebook_path}'")
        return

    with open(notebook_path, 'r', encoding='utf-8') as f:
        notebook = json.load(f)

    toc_markdown = "### **Table of Contents**\n"
    for cell in notebook.get('cells', []):
        if cell.get('cell_type') == 'markdown':
            for line in cell.get('source', []):
                if line.strip().startswith('#'):
                    level = line.count('#')
                    title = line.strip('#').strip()
                    link = title.lower().replace(' ', '-').strip('-.()')
                    indent = '  ' * (level - 1)
                    toc_markdown += f"{indent}* [{title}](#{link})\n"

    print("\n--- ✅ Copy the Markdown below and paste"
          "it into a new markdown cell ---\n")
    print(toc_markdown)


if __name__ == "__main__":
    # Example usage
    notebook_path = 'notebook.ipynb'  # Replace with your notebook path
    generate_toc_from_notebook(notebook_path)



--- ✅ Copy the Markdown below and pasteit into a new markdown cell ---

### **Table of Contents**
* [Imports](#imports)
* [Read in data](#read-in-data)

